In [3]:
import sqlite3

conn = sqlite3.connect("badge_attendance.db")
cursor = conn.cursor()


In [1]:
%load_ext sql

In [2]:
%sql sqlite:///badge_attendance.db

In [4]:
%%sql
CREATE TABLE IF NOT EXISTS fact_attendance_daily (
    fact_id INTEGER PRIMARY KEY AUTOINCREMENT,
    badge_id TEXT,
    person_type TEXT,
    date_id INTEGER,
    total_hours REAL,
    session_count INTEGER
);


 * sqlite:///badge_attendance.db
Done.


[]

In [6]:
%%sql
INSERT INTO fact_attendance_daily (
    badge_id,
    person_type,
    date_id,
    total_hours,
    session_count
)
WITH ordered_events AS (
    SELECT
        badge_id,
        person_type,
        event_time,
        event_type,
        LEAD(event_time) OVER (
            PARTITION BY badge_id
            ORDER BY event_time
        ) AS next_time
    FROM src_badge_events
),
sessions AS (
    SELECT
        badge_id,
        person_type,
        event_time AS in_time,
        next_time AS out_time
    FROM ordered_events
    WHERE event_type = 'IN'
),
session_durations AS (
    SELECT
        badge_id,
        person_type,
        DATE(in_time) AS event_date,
        (julianday(out_time) - julianday(in_time)) * 24 AS session_hours
    FROM sessions
    WHERE out_time IS NOT NULL
)
SELECT
    s.badge_id,
    s.person_type,
    c.date_id,
    ROUND(SUM(s.session_hours), 2) AS total_hours,
    COUNT(*) AS session_count
FROM session_durations s
JOIN dim_calendar c
    ON s.event_date = c.full_date
GROUP BY s.badge_id, s.person_type, c.date_id;


 * sqlite:///badge_attendance.db
302261 rows affected.


[]

In [7]:
%%sql
SELECT COUNT(*) FROM fact_attendance_daily;


 * sqlite:///badge_attendance.db
Done.


COUNT(*)
302261


In [9]:
%%sql
SELECT * FROM fact_attendance_daily LIMIT 10;


 * sqlite:///badge_attendance.db
Done.


fact_id,badge_id,person_type,date_id,total_hours,session_count
1,BADGE000000,student,20240901,2.75,2
2,BADGE000000,student,20240902,4.57,3
3,BADGE000000,student,20240903,2.65,1
4,BADGE000000,student,20240904,0.77,1
5,BADGE000000,student,20240905,0.32,1
6,BADGE000000,student,20240906,1.45,1
7,BADGE000000,student,20240907,3.12,1
8,BADGE000000,student,20240909,1.4,1
9,BADGE000000,student,20240910,4.9,2
10,BADGE000000,student,20240912,1.3,1


In [10]:
%%sql
SELECT
    f.badge_id,
    c.full_date,
    f.total_hours
FROM fact_attendance_daily f
JOIN dim_calendar c
    ON f.date_id = c.date_id
LIMIT 10;


 * sqlite:///badge_attendance.db
Done.


badge_id,full_date,total_hours
BADGE000000,2024-09-01,2.75
BADGE000000,2024-09-02,4.57
BADGE000000,2024-09-03,2.65
BADGE000000,2024-09-04,0.77
BADGE000000,2024-09-05,0.32
BADGE000000,2024-09-06,1.45
BADGE000000,2024-09-07,3.12
BADGE000000,2024-09-09,1.4
BADGE000000,2024-09-10,4.9
BADGE000000,2024-09-12,1.3


In [11]:
%%sql
SELECT
    s.name,
    SUM(f.total_hours) AS total_hours
FROM fact_attendance_daily f
JOIN dim_students s
    ON f.badge_id = s.badge_id
GROUP BY s.name
ORDER BY total_hours DESC
LIMIT 10;


 * sqlite:///badge_attendance.db
Done.


name,total_hours
Anthony Johnson,893.15
Jeremy Jones,891.53
Michael King,884.03
Brandi Reed,855.52
Joshua Hamilton,854.69
Patricia King,853.4
Nicholas Thompson,836.42
Jennifer Smith,834.44
Michael Johnson,831.12
Nicholas Smith,830.36


In [16]:
%%sql
SELECT COUNT(*) FROM dim_teachers;


 * sqlite:///badge_attendance.db
Done.


COUNT(*)
150


In [17]:
%%sql
SELECT COUNT(*)
FROM fact_attendance_daily f
JOIN dim_teachers t
ON f.badge_id = t.badge_id;


 * sqlite:///badge_attendance.db
Done.


COUNT(*)
29592


In [18]:
%%sql
SELECT
    t.name AS teacher_name,
    t.department AS subject,
    SUM(f.total_hours) AS total_hours
FROM fact_attendance_daily f
JOIN dim_teachers t
ON f.badge_id = t.badge_id
GROUP BY t.name, t.department
ORDER BY total_hours DESC
LIMIT 10;


 * sqlite:///badge_attendance.db
Done.


teacher_name,subject,total_hours
Terry Mendez,Law,798.64
Zachary Chan,Arts,786.51
Leslie Holmes,Engineering,777.41
Tara Sullivan,Arts,770.86
Paula Frazier,Engineering,770.46
Amy Caldwell,Science,770.33
Michael Spencer,Law,767.63
Mark Andrews,Engineering,764.47
Diana Lopez DDS,Business,759.13
Sara Butler,Engineering,758.96


In [35]:
%%sql
SELECT
    t.name,
    SUM(f.total_hours) AS total_hours
FROM fact_attendance_daily f
JOIN dim_teachers t ON f.badge_id = t.badge_id
JOIN dim_calendar c ON f.date_id = c.date_id
WHERE c.year = 2024 AND c.month = 9
GROUP BY t.name
ORDER BY total_hours DESC
LIMIT 10;


 * sqlite:///badge_attendance.db
Done.


name,total_hours
Brandi Roberts,128.89
Sara Butler,125.75
Jacob Cervantes,124.3
Zachary Chan,122.83
Tyler Martinez,121.76
Elizabeth Marshall,121.46
Audrey Moore,120.98
Thomas Campbell,120.41
Jeffrey Roth,119.2
Crystal Palmer,118.79
